# Risk aversion

Terry Tao recently shared some thoughts about risk aversion [on Mastodon](https://mathstodon.xyz/@tao/113479000564381543).

His example is easy to model in memo. First, some imports and boilerplate…

In [1]:
from memo import memo
import jax.numpy as np
import jax
from enum import IntEnum

class Action(IntEnum):  # two types of actions
    Safe = 0
    Bold = 1

# Outcome space: unit normal, support truncated to -10 to 10, discretized to 101 cells
Outcome = np.linspace(-10, 10, 101)
from jax.scipy.stats.norm import pdf as normpdf
normpdf = jax.jit(normpdf)

An agent's utility depends on the type of action and the outcome.

In [2]:
@jax.jit
def utility(a, o):
    means = np.array([5, 9])  # safe, bold
    stdvs = np.array([3, 10]) # safe, bold
    return means[a] + stdvs[a] * o

Now in memo, we can model an agent who minimizes their "value at risk", which is defined by Terry as $\sqrt{\text{Var}[u]} - E[u]$ for utility $u$.

In [3]:
@memo
def model[a: Action]():
    terry: chooses(a in Action, to_minimize=imagine[  # terry minimizes "value at risk"
        world: chooses(o in Outcome, wpp=normpdf(o)),  # outcome of action ~ N(0, 1)
        # value risk:
        Var[utility(a, world.o)]**0.5 - E[utility(a, world.o)]
    ])
    return Pr[terry.a == a]
for a in Action: print(f'P({a.name}) = {model()[a]}')

P(Safe) = 1.0
P(Bold) = 0.0


Terry chooses the safe action.

Now what happens if we introduce an external "shock" factor with mean 5 and variance 10?

In [4]:
@jax.jit
def utility(a, o, shock):
    mean = np.array([5, 9])
    stdv = np.array([3, 10])
    return mean[a] + stdv[a] * o + (-5 + 10 * shock)

@memo
def model[a: Action]():
    terry: chooses(a in Action, to_minimize=imagine[  # terry minimizes "value at risk"
        world: chooses(o in Outcome, wpp=normpdf(o)),  # outcome of action ~ N(0, 1)
        world: chooses(s in Outcome, wpp=normpdf(s)),  # external "shock" factor ~ N(0, 1)
        # value risk:
        Var[utility(a, world.o, world.s)]**0.5 - E[utility(a, world.o, world.s)]
    ])
    return Pr[terry.a == a]

for a in Action: print(f'P({a.name}) = {model()[a]}')

P(Safe) = 0.0
P(Bold) = 1.0


Now Terry prefers the risky action!